In [9]:
import sys
from os import path
import re

In [10]:
code = """
data race;
pr = probnorm(-15/sqrt(325));
run;

%macro date_loop(start,end);
   %let start=%sysfunc(inputn(&start,anydtdte9.));
   %let end=%sysfunc(inputn(&end,anydtdte9.));
   %let dif=%sysfunc(intck(month,&start,&end));
     %do i=0 %to &dif;
      %let date=%sysfunc(intnx(month,&start,&i,b),date9.);
      %put &date;
     %end;
   %mend date_loop;

   %date_loop(01jul2015,01feb2016)
   
proc print data=race;
var pr;
run;


proc iml;
FF = FINV(0.05/32,2,29);
print FF;
quit;

%macro list_files(dir,ext);
  %local filrf rc did memcnt name i;
  %let rc=%sysfunc(filename(filrf,&dir));
  %let did=%sysfunc(dopen(&filrf));      

   %if &did eq 0 %then %do; 
    %put Directory &dir cannot be open or does not exist;
    %return;
  %end;

   %do i = 1 %to %sysfunc(dnum(&did));   

   %let name=%qsysfunc(dread(&did,&i));

      %if %qupcase(%qscan(&name,-1,.)) = %upcase(&ext) %then %do;
        %put &dir\&name;
      %end;
      %else %if %qscan(&name,2,.) = %then %do;        
        %list_files(&dir\&name,&ext)
      %end;

   %end;
   %let rc=%sysfunc(dclose(&did));
   %let rc=%sysfunc(filename(filrf));     

%mend list_files;
%list_files(c:\temp,sas)


data chisq;
input df;
chirat = cinv(.995,df)/cinv(.005,df);
datalines;
20
21
22
23
24
25
26
27
28
29
30
;
run;
 
proc print data=chisq;
var df chirat;
run;
 
proc plot data=chisq;
plot chirat*df;
run;
"""

In [ ]:
(data\b|proc|%macro).*\n(.*)(\n|\t|\n\t)?(.*)(run;|quit;|%mend)
(proc).*\n(.*)(\n|\t|\n\t)?(.*)(quit;)
(?m)(^proc.*((\n|\t|\n\t)[ ](proc.*)(.*))*)

In [ ]:
comment = '/*'

In [ ]:
def sas2nb(sas_str):
    if sas_str.startswith(comment):
        pass
    cells = []
    chunks = sas_str.split('\n\n%s' % header_comment)
        

In [1]:
notebook = {
        'cells': cells,
        'metadata': {
            'anaconda-cloud': {},
            'kernelspec': {
                'display_name': 'SAS',
                'language': 'sas',
                'name': 'sas'},
            'language_info': {
                'codemirror_mode': {'name': 'sas'},
                'file_extension': '.sas',
                'name': 'sas',
                'nbconvert_exporter': 'python',
                }
        },
        'nbformat': 4,
        'nbformat_minor': 4
    }

NameError: name 'cells' is not defined

In [2]:
def convert(in_file, out_file):
    _, in_ext = path.splitext(in_file)
    _, out_ext = path.splitext(out_file)

    if in_ext == '.ipynb' and out_ext == '.sas':
        with open(in_file, 'r', encoding='utf-8') as f:
            notebook = json.load(f)
        py_str = nb2py(notebook)
        with open(out_file, 'w', encoding='utf-8') as f:
            f.write(py_str)

    elif in_ext == '.sas' and out_ext == '.ipynb':
        with open(in_file, 'r', encoding='utf-8') as f:
            sas_str = f.read()
        notebook = sas2nb(py_str)
        with open(out_file, 'w', encoding='utf-8') as f:
            json.dump(notebook, f, indent=2)

    else:
        raise(Exception('Extensions must be .ipynb and .sas or vice versa'))

In [7]:
def main():
    argv = sys.argv
    if len(argv) < 3:
        print('Usage: ipynb-sas-convert in.ipynb out.sas')
        print('or:    ipynb-sas-convert in.sas out.ipynb')
        sys.exit(1)

    convert(in_file=argv[1], out_file=argv[2])


if __name__ == '__main__':
    main()

Exception: Extensions must be .ipynb and .sas or vice versa